<a href="https://colab.research.google.com/github/MichaelKru92/Projekt-ML-Modelierung/blob/main/Schritt_5_Datentransformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Datentransformation**

**Fehlende Eingaben:**

Der Umgang mit Null-Werten in den Rohdaten wurde im Hinblick auf die Fragestellung zur Vorhersage der Gesundheitskosten für die betroffenen Parameter geprüft mit folgenden Fazit:

*   STOP: Nullwerte Tabellen übergreifend beibehalten
*   REASONCODE, REASONDESCRIPTION: Entfernen der betroffenen Zeilen in Careplans-Tabelle
*   DRIVERS, PASSPORT, PREFIX, SUFFIX, MAIDEN, MARITAL: Entfernen der betroffenen Zeilen in Patients-Tabelle
*   REASONCODE, REASONDESCRIPTION: Entfernen der Spalten der Encounters-Tabelle
*   REASONCODE, REASONDESCRIPTION: Entfernen der Spalten der Medications-Tabelle
*   ENCOUNTER: Entfernen der Zeile in der Observations-Tabelle

**Zeitangaben:**

Datumsangaben (Spalten: START, STOP) werden vereinheitlicht.